In [ ]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from ydata_profiling import ProfileReport, compare
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.figure_factory as ff
from IPython.display import clear_output

from quick_pp.database.objects import Project
from quick_pp.database.db_connector import DBConnector

db_conn = DBConnector()

project_name = "MOCK_carbonate"
with db_conn.get_session() as db_session:
    # Load well from saved file
    project = Project(db_session, name=project_name)
    df = project.get_all_data()
    print(project.get_well_names())

In [ ]:
folder = f"outputs/{project_name}/"
if not os.path.exists(folder):
    os.makedirs(folder)

## Overall Profiling Report

In [ ]:
profiles = ProfileReport(df, title="Profiling Report")
profiles.to_file(rf"outputs\{project_name}\{project_name}_overall_profiling.html")
clear_output()

## Individual Well Profiling Report

In [ ]:
for well_name, well in df.groupby("WELL_NAME"):
    profile = ProfileReport(well.drop(columns=['WELL_NAME']), title=f"{well_name} Profiling Report",
                            explorative=True)
    print(f"Generating {well_name} profiling report...")
    profile.to_file(rf"outputs\{project_name}\{well_name.replace('/', '_')}_profiling_report.html")
clear_output()

## Compare selected wells with the rest of the wells

In [ ]:
well_name = "HW-3"
well = df[df["WELL_NAME"] == well_name][['GR', 'RT', 'NPHI', 'RHOB']]
profile = ProfileReport(well, title=f"{well_name} Profiling Report", explorative=True)

others = df[df["WELL_NAME"] != well_name][['GR', 'RT', 'NPHI', 'RHOB']]
other_profiles = ProfileReport(others, title=f"Other Wells Profiling Report", explorative=True)

compare([profile, other_profiles]).to_file(rf"outputs\{project_name}\{well_name}_compare.html")
clear_output()

## Compare distribution of selected variables by wells

In [ ]:
for var in ['GR', 'RT', 'NPHI', 'RHOB']:
    sns.kdeplot(data=df.dropna(subset=var), x=var, hue='WELL_NAME',
                log_scale=True if var == 'RT' else False, fill=True, legend=True)
    plt.title(f"{var} Distribution")
    sns.move_legend(plt.gca(), "upper left", bbox_to_anchor=(1, 1))
    plt.show()